In [22]:
import os
import random
from datetime import datetime
from collections import Counter

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

from utils import misc, preprocessing, evaluation_metrics
from call_backs import TestEmbeddingCallback
from custom_layers.arcface_loss import ArcMarginProduct

# to access cifar100
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

### Config

In [23]:
class config:
    # GENERAL
    RANDOM_SEED = 5
    TENSOR_LOG_DIR = 'logs'
    SAVE_DIR = 'saved_models'

    # DATA
    INPUT_SIZE = (32,32,3)
    NUM_CLASSES = 100 # holding out 5 classes from cifar100

    # MODEL
    OUTPUT_EMB = 64
    MIDDLE_EMB = 256

    # TRAINING
    EPOCHS = 60
    BATCH_SIZE = 32
    LR = .0005

misc.seed_everything(config.RANDOM_SEED)

### Load dataset

In [24]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

x_train = tf.convert_to_tensor(x_train)
y_train = tf.convert_to_tensor(y_train)

# images are of correct input_size
print('x_train shape:', x_train.get_shape())
assert x_train.get_shape()[-3:] == config.INPUT_SIZE

print('y_train shape:', y_train.get_shape())
# y_train has only 1 label per item in tensor
assert y_train.get_shape()[-1:] == 1

# num images == num labels
assert y_train.get_shape()[0] == x_train.get_shape()[0]



x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)


In [25]:
class_count = Counter(np.array(tf.reshape(y_train, [y_train.get_shape()[0],])))
print(class_count)

Counter({19: 500, 29: 500, 0: 500, 11: 500, 1: 500, 86: 500, 90: 500, 28: 500, 23: 500, 31: 500, 39: 500, 96: 500, 82: 500, 17: 500, 71: 500, 8: 500, 97: 500, 80: 500, 74: 500, 59: 500, 70: 500, 87: 500, 84: 500, 64: 500, 52: 500, 42: 500, 47: 500, 65: 500, 21: 500, 22: 500, 81: 500, 24: 500, 78: 500, 45: 500, 49: 500, 56: 500, 76: 500, 89: 500, 73: 500, 14: 500, 9: 500, 6: 500, 20: 500, 98: 500, 36: 500, 55: 500, 72: 500, 43: 500, 51: 500, 35: 500, 83: 500, 33: 500, 27: 500, 53: 500, 92: 500, 50: 500, 15: 500, 18: 500, 46: 500, 75: 500, 38: 500, 66: 500, 77: 500, 69: 500, 95: 500, 99: 500, 93: 500, 4: 500, 61: 500, 94: 500, 68: 500, 34: 500, 32: 500, 88: 500, 67: 500, 30: 500, 62: 500, 63: 500, 40: 500, 26: 500, 48: 500, 79: 500, 85: 500, 54: 500, 44: 500, 7: 500, 12: 500, 2: 500, 41: 500, 37: 500, 13: 500, 25: 500, 10: 500, 57: 500, 5: 500, 60: 500, 91: 500, 3: 500, 58: 500, 16: 500})


In [26]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(preprocessing.normalize).map(preprocessing.arcface_format).batch(config.BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(preprocessing.normalize).map(preprocessing.arcface_format).batch(config.BATCH_SIZE)

### Define model 

In [27]:
# from custom_layers.arcface_loss import ArcMarginProduct
from custom_layers.subcenter_arcface_loss import SubcenterArcMarginProduct as ArcMarginProduct
# allows 2 inputs and 2 outputs

def get_debug_model(s = 10, m = .25, k = 3):
 #------------------
    # Definition of placeholders
    inp = tf.keras.layers.Input(shape = config.INPUT_SIZE, name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')

    # Definition of layers
    
    #TODO: reasearch filters, get better understanding
    layer_conv1 = tf.keras.layers.Conv2D(filters = 24, kernel_size = (2,2), input_shape = config.INPUT_SIZE, activation ='relu')
    layer_pool1 = tf.keras.layers.MaxPool2D((2,2))
    layer_conv2 = tf.keras.layers.Conv2D(filters = 12, kernel_size = (2,2), activation ='relu')
    layer_pool2 = tf.keras.layers.MaxPool2D((2,2))
    layer_flatten = tf.keras.layers.Flatten()
    layer_dense1 = tf.keras.layers.Dense(config.MIDDLE_EMB)
    layer_dense2 = tf.keras.layers.Dense(config.NUM_CLASSES)
    layer_arcface = ArcMarginProduct(n_classes=config.NUM_CLASSES, s=s, m=m, k=k)
    layer_softmax = tf.keras.layers.Softmax(dtype='float16', name='head_output')

    if config.MIDDLE_EMB != config.OUTPUT_EMB:
        layer_adaptive_pooling = tfa.layers.AdaptiveAveragePooling1D(config.OUTPUT_EMB)
    else:
        layer_adaptive_pooling = tf.keras.layers.Lambda(lambda x: x)  # layer with no operation

    #------------------
    # Definition of entire model
    backbone_output = layer_conv1(inp)
    backbone_output = layer_pool1(backbone_output)
    backbone_output = layer_conv2(backbone_output)
    backbone_output = layer_pool2(backbone_output)
    embed = layer_flatten(backbone_output)
    embed = layer_dense1(embed)
    
    # Training head
    # head_output = layer_dense2(embed)
    head_output = layer_arcface((embed,label))
    head_output = layer_softmax(head_output)
    
    # Inference
    emb_output = layer_adaptive_pooling(embed)

    model = tf.keras.models.Model(inputs = [(inp, label)], outputs = [head_output, emb_output]) # whole architecture

    return model

In [28]:
debug_model = get_debug_model(s=32, m=.15, k=2)
debug_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inp1 (InputLayer)              [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 31, 31, 24)   312         ['inp1[0][0]']                   
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 15, 15, 24)  0           ['conv2d_2[0][0]']               
                                                                                                  
 conv2d_3 (Conv2D)              (None, 14, 14, 12)   1164        ['max_pooling2d_2[0][0]']        
                                                                                            

In [29]:
debug_model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate = config.LR),
        loss = {'head_output':tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)},
        metrics = {'head_output':[tf.keras.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3)]},
        )

steps_per_epoch = len(train_dataset) // config.BATCH_SIZE  // 20     # "//20" means that the lr is update every 0.1 epoch.
validation_steps = len(test_dataset) // config.BATCH_SIZE
if len(test_dataset) % config.BATCH_SIZE != 0:
    validation_steps += 1
print(steps_per_epoch, validation_steps)

2 10


### Callbacks

In [30]:
# tensorboard

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,
                        #  write_graph=True,
                        #  write_images=True,
                        update_freq='epoch',
                        #  profile_batch=2,
                        #  embeddings_freq=1
                        )

#emb_callback = call_backs.EmbeddingCallback(x_test, y_test, save_dir = log_dir+'/emb/', embedding_dim = config.OUTPUT_EMB)
emb_callback = TestEmbeddingCallback(x_test, y_test, 'logs/emb/')

### Training

In [31]:
history = debug_model.fit(
        train_dataset,
        epochs=config.EPOCHS,
        validation_steps = validation_steps,
        validation_data = test_dataset,
        verbose=1,
        callbacks=[tensorboard_callback, emb_callback]
    )

Epoch 1/60
313/313 [==============================] - 5s 16ms/step
Competition score was 0.11392
1563/1563 [==============================] - 79s 49ms/step - loss: 8.3479 - head_output_loss: 8.3479 - head_output_sparse_categorical_accuracy: 0.0000e+00 - head_output_sparse_top_k_categorical_accuracy: 8.0000e-05 - val_loss: 7.9910 - val_head_output_loss: 7.9910 - val_head_output_sparse_categorical_accuracy: 0.0000e+00 - val_head_output_sparse_top_k_categorical_accuracy: 0.0000e+00
Epoch 2/60
313/313 [==============================] - 3s 9ms/step loss: 7.9495 - head_output_loss: 7.9495 - head_out
Competition score was 0.115
1563/1563 [==============================] - 80s 51ms/step - loss: 7.9491 - head_output_loss: 7.9491 - head_output_sparse_categorical_accuracy: 8.0000e-05 - head_output_sparse_top_k_categorical_accuracy: 6.2000e-04 - val_loss: 7.8816 - val_head_output_loss: 7.8816 - val_head_output_sparse_categorical_accuracy: 0.0000e+00 - val_head_output_sparse_top_k_categorical_accur

In [32]:
print("Test examples:",len(x_test))
pred_class, pred_emb = debug_model.predict((x_test, y_test))  # I don't like this, (hacky way would be y test of -1s if non given)

true_class = y_test

Test examples: 10000
313/313 [==============================] - 4s 12ms/step


In [33]:
embedding_data, tree = evaluation_metrics.test_embeddings(true_class,pred_class,pred_emb,config.NUM_CLASSES)

emb_df = pd.DataFrame(embedding_data)
emb_df.head()

,annoy_idx,true_class,pred_class,embedding
0,0,49,95,"[43.73854, 15.851908, -1.7649889, 3.4214182, -..."
1,1,33,13,"[15.373831, -13.585405, -20.034208, -21.310646..."
2,2,72,13,"[75.07383, 94.89427, -40.70049, 1.7230673, -77..."
3,3,51,13,"[60.414726, 101.26077, -44.709564, -34.36889, ..."
4,4,71,62,"[4.781587, 0.46057606, -26.04817, -20.19368, -..."


In [34]:
emb_df['length']=emb_df['embedding'].apply(evaluation_metrics.dist_to_origin)
emb_df['normed_embeddings']=emb_df['embedding']/emb_df['length']

In [35]:
tree.build(20)

True

In [36]:
emb_df['nearest_neighbors'] = emb_df['annoy_idx'].apply(lambda row: evaluation_metrics.n_neighbors(row,tree))
emb_df['neighbor_classes'] = emb_df.apply(lambda row: evaluation_metrics.neighbor_classes(row,emb_df,true_classes=True), axis=1)
emb_df['neighbor_pred_classes'] = emb_df.apply(lambda row: evaluation_metrics.neighbor_classes(row,emb_df,true_classes=False), axis=1)
emb_df['matching_neighbors'] = emb_df.apply(lambda row: evaluation_metrics.matching_neighbors(row,true_classes=True), axis=1)
emb_df['matching_neighbor_preds'] = emb_df.apply(lambda row: evaluation_metrics.matching_neighbors(row,true_classes=False), axis=1)

In [37]:
emb_df.head()

,annoy_idx,true_class,pred_class,embedding,length,normed_embeddings,nearest_neighbors,neighbor_classes,neighbor_pred_classes,matching_neighbors,matching_neighbor_preds
0,0,49,95,"[43.73854, 15.851908, -1.7649889, 3.4214182, -...",365.866425,"[0.11954784, 0.043327037, -0.0048241345, 0.009...","[1642, 3298, 9786, 9528, 212]","[49, 39, 39, 95, 49]","[95, 95, 95, 39, 95]",2,4
1,1,33,13,"[15.373831, -13.585405, -20.034208, -21.310646...",189.496704,"[0.081129804, -0.07169204, -0.105723254, -0.11...","[9914, 2574, 608, 2521, 180]","[39, 64, 46, 73, 42]","[11, 98, 98, 95, 11]",0,0
2,2,72,13,"[75.07383, 94.89427, -40.70049, 1.7230673, -77...",401.948578,"[0.18677472, 0.23608561, -0.10125795, 0.004286...","[7489, 9460, 7842, 5636, 3502]","[51, 35, 48, 54, 45]","[82, 11, 13, 11, 13]",0,2
3,3,51,13,"[60.414726, 101.26077, -44.709564, -34.36889, ...",436.840546,"[0.13829927, 0.2318026, -0.10234756, -0.078676...","[8749, 8820, 9214, 3742, 1489]","[13, 51, 85, 45, 40]","[20, 13, 13, 20, 13]",1,3
4,4,71,62,"[4.781587, 0.46057606, -26.04817, -20.19368, -...",205.809570,"[0.023233065, 0.0022378748, -0.12656443, -0.09...","[6399, 9567, 996, 9734, 3031]","[36, 55, 50, 74, 94]","[53, 82, 39, 53, 53]",0,0


In [38]:
evaluation_metrics.competition_score(emb_df,5)

0.1067

In [39]:
emb_df['correct_prediction'] = emb_df['true_class']==emb_df['pred_class']

In [40]:
print(emb_df['correct_prediction'].value_counts())

False    10000
Name: correct_prediction, dtype: int64


In [41]:
emb_df['matching_neighbor_preds'].value_counts()

1    2214
0    2105
2    2047
3    1786
4    1208
5     640
Name: matching_neighbor_preds, dtype: int64

In [42]:
emb_df['matching_neighbors'].value_counts()

0    6858
1    1873
2     705
3     299
4     170
5      95
Name: matching_neighbors, dtype: int64